sfsdfsdおれ

In [6]:
%pip install pandas
%pip install tqdm
%pip install re

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd

# データAとデータBのCSVファイルのパス
data_a_path = '../data/original_data/race_table/combined.csv'  # データAのCSVファイルパス
data_b_path = '../data/merged_data/merged_sorted.csv'  # データBのCSVファイルパス

# CSVファイルからデータを読み込む
data_a = pd.read_csv(data_a_path,low_memory=False)
data_b = pd.read_csv(data_b_path)
data_a=data_a.dropna()
data_b=data_b.dropna()
data_a=data_a.drop(columns=['first_horses', 'second_horses', 'third_horses', 'Time', 'Jockey', 'Trainer'])
# Code列でデータを結合 (data_bにdata_aの情報を追加)
merged_data = pd.merge(data_b, data_a, on='Code', how='left')

#結果の確認
merged_data.to_csv('../notebook/data-shin/merged_data_final.csv', index=False)
for i in merged_data.columns:
    # 各列のユニークな値とその出現回数を取得
    value_counts = df[i].value_counts()  # 各値とそのカウント
    print(f"{i}: {value_counts}")


/tmp/ipykernel_1125/1400195009.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data_b = pd.read_csv(data_b_path)


特徴量の析出

In [9]:
import pandas as pd
import re
from tqdm import tqdm

file_path = '../notebook/data-shin/merged_data_final.csv'
# CSVファイルの読み込み
df = pd.read_csv(file_path)
# 12桁のコードのみを抽出
df = df[df['Code'].astype(str).str.match(r'^\d{12}$')]

# 正規表現の事前コンパイル（前後にある任意の空白を無視）
time_pattern1 = re.compile(r'\s*(\d+):(\d+\.\d+)\s*')
time_pattern2 = re.compile(r'\s*(\d+):(\d+):(\d+)\s*')
sex_age_pattern = re.compile(r'\s*([牡牝セ])(\d+)\s*')
distance_pattern = re.compile(r'\s*([ダ芝障])(\d+)\s*')
condition_pattern = re.compile(r'\s*([良重稍不])\s*')
weather_pattern = re.compile(r'\s*(晴|曇|小雨|雨|雪|小雪)\s*')
kinryou_pattern = re.compile(r'(\d+)\(([\+\-]?\d+)\)')


# 時間を秒に変換する関数 (ベクトル化)
def time_to_seconds(time_series):
    result = []
    for time_str in time_series:
        if isinstance(time_str, str):
            match1 = time_pattern1.match(time_str)
            if match1:
                minutes, seconds = int(match1.group(1)), float(match1.group(2))
                result.append(minutes * 60 + seconds)
                continue
            match2 = time_pattern2.match(time_str)
            if match2:
                minutes, seconds = int(match2.group(1)), float(match2.group(2))
                result.append(minutes * 60 + seconds)
                continue
        result.append(None)
    return result


# 性別と年齢を抽出する関数（ベクトル化）
def extract_sex_age_column(sex_age_series):
    sexes, ages = [], []
    for text in sex_age_series:
        match = sex_age_pattern.match(text) if isinstance(text, str) else None
        if match:
            sex = match.group(1)
            if sex in ['牡', '牝', 'セ']:  # 性別が牡、牝、セのみを許可
                sexes.append(sex)
                ages.append(int(match.group(2)))
            else:
                sexes.append(None)
                ages.append(None)
        else:
            sexes.append(None)
            ages.append(None)
    return pd.DataFrame({'Sex': sexes, 'Age': ages})


# 距離を抽出する関数（ベクトル化）
def extract_distance_column(distance_series):
    statuses, distances = [], []
    for text in distance_series:
        match = distance_pattern.match(text) if isinstance(text, str) else None
        if match:
            statuses.append(match.group(1))
            distances.append(int(match.group(2)))
        else:
            statuses.append(None)
            distances.append(None)
    return pd.DataFrame({'Sta': statuses, 'Dis': distances})


# 条件を抽出する関数（ベクトル化）
def extract_condition_column(condition_series):
    conditions = []
    
    for text in condition_series:
        match = condition_pattern.match(text) if isinstance(text, str) else None
        if match:
            conditions.append(match.group(1))
        else:
            conditions.append(None)
    return pd.Series(conditions)


# 天気を抽出する関数（ベクトル化）
def extract_weather_column(weather_series):
    weathers = []
    for text in weather_series:
        match = weather_pattern.match(text) if isinstance(text, str) else None
        if match:
            weathers.append(match.group(1))
        else:
            weathers.append(None)
    return pd.Series(weathers)


# 体重の変化を抽出する関数
def extract_weight_change(weight_series):
    weights, changes = [], []
    for text in weight_series:
        match = re.match(r'(\d+)\(([\+\-]?\d+)\)', text) if isinstance(text, str) else None
        if match:
            weights.append(int(match.group(1)))  # 体重
            changes.append(int(match.group(2)))  # 変化量
        else:
            weights.append(None)
            changes.append(None)
    return pd.DataFrame({'Weight': weights, 'Change': changes})



# tqdmの進捗バーを使用して変換
tqdm.pandas()
df['Time'] = time_to_seconds(df['Time'].values)

# SexとAgeの列に分割して格納
sex_age_df = extract_sex_age_column(df['Sex/Age'].values)
df[['Sex', 'Age']] = sex_age_df

# StaとDisの列に分割して格納
distance_df = extract_distance_column(df['Distance'].values)
df[['Sta', 'Dis']] = distance_df

# Conditionの列に分割して格納
df['Condition'] = extract_condition_column(df['Condition'].values)

# Weatherの列に分割して格納
df['Weather'] = extract_weather_column(df['Weather'].values)
# 新しいカラム 'Horse Weight' を追加して処理
weight_change_df = extract_weight_change(df['Horse Weight'].values)
df[['Weight', 'Change']] = weight_change_df

# NaN 値が含まれている行をフィルタリング
df = df.dropna(subset=['Sex', 'Age', 'Sta', 'Dis', 'Condition', 'Weather'])

# 不要な列を削除
df = df.drop('Sex/Age', axis=1)
df = df.drop('Distance', axis=1)
df=df.drop('Horse Weight',axis=1)
# フィルタリングしたデータをCSVに保存
df.to_csv('../notebook/data-shin/filtered_data.csv', index=False)

/tmp/ipykernel_1125/1133966646.py:7: DtypeWarning: Columns (1,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [11]:
import pandas as pd
import numpy as np

# データの読み込み
df = pd.read_csv('../notebook/data-shin/filtered_data.csv')

# 必要なカテゴリ変数の数値化
# 方法1: 文字列カテゴリを整数に変換
df['Horse Name'] = pd.factorize(df['Horse Name'])[0]
df['Jockey'] = pd.factorize(df['Jockey'])[0]
df['Trainer'] = pd.factorize(df['Trainer'])[0]
df['Banushi'] = pd.factorize(df['Banushi'])[0]
df['Track'] = pd.factorize(df['Track'])[0]
df['Weather'] = pd.factorize(df['Weather'])[0]
df['Sex'] = pd.factorize(df['Sex'])[0]

# 方法2: 日付データをカテゴリ型に変換する例
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].map(lambda date: date.toordinal())  # 日付を数値化（連続した数値）

# Weight列の処理 - '5(降)'のような文字列から数値のみを抽出
df['Weight'] = df['Weight'].apply(lambda x: float(str(x).split('(')[0]) if pd.notnull(x) else np.nan)
df['Change'] = df['Change'].apply(lambda x: float(x) if pd.notnull(x) else np.nan)

# 数値型の列のみを選択して相関係数行列を計算
numeric_cols = df.select_dtypes(include=[np.number]).columns
corr_matrix = df[numeric_cols].corr()

# 結果を表示
print(corr_matrix)


/tmp/ipykernel_1125/1704768020.py:5: DtypeWarning: Columns (1,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../notebook/data-shin/filtered_data.csv')


                  Code  Frame Rank  Horse Number  Horse Name   Kinryou  \
Code          1.000000    0.012593     -0.065354   -0.941328  0.134078   
Frame Rank    0.012593    1.000000      0.883729   -0.011883 -0.000154   
Horse Number -0.065354    0.883729      1.000000    0.052716  0.028397   
Horse Name   -0.941328   -0.011883      0.052716    1.000000 -0.132046   
Kinryou       0.134078   -0.000154      0.028397   -0.132046  1.000000   
Jockey       -0.672068   -0.005887      0.023485    0.619328 -0.078536   
Time         -0.084425    0.000284      0.019110    0.075380  0.309331   
Nobori        0.079946    0.019357     -0.086097   -0.067430 -0.310887   
Ninki        -0.050897    0.001498      0.078591    0.052998 -0.061918   
Trainer      -0.664200   -0.003077      0.041090    0.618055 -0.102605   
Banushi      -0.620062   -0.004301      0.013293    0.593453 -0.110018   
Date          0.999226    0.012155     -0.062054   -0.943088  0.134428   
Track        -0.351573   -0.023983    